# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.58,67,28,0.83,TF,1738602701
1,1,waitangi,-43.9535,-176.5597,14.90,90,57,4.47,NZ,1738602809
2,2,margaret river,-33.9500,115.0667,15.23,81,0,6.10,AU,1738602810
3,3,chonchi,-42.6128,-73.8092,14.51,54,100,1.66,CL,1738602516
4,4,constitucion,-35.3333,-72.4167,21.64,53,10,7.43,CL,1738602812


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    size="Humidity",
    hover_cols=["City", "Humidity"],
    tiles="OSM",
    color="City",
    alpha=0.6,
    width=1000,
    height=600
)
        
# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df[
    (city_data_df["Max Temp"] > 22) &
    (city_data_df["Lat"].between(-2, 20)) &
    (city_data_df["Cloudiness"] < 20) &
    (city_data_df["Wind Speed"] < 3)
]

# Drop any rows with null values
ideal_city = ideal_city.dropna()

# Display sample data
ideal_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
120,120,coahuayana de hidalgo,18.7000,-103.6583,31.39,31,0,2.01,MX,1738602959
201,201,kottapalli,18.4833,79.1000,25.78,31,5,2.34,IN,1738603062
531,531,palasa,18.7667,84.4167,22.85,75,3,1.19,IN,1738603469
543,543,san luis de la loma,17.2706,-100.8939,29.04,54,0,2.35,MX,1738603484
551,551,beli,7.8587,10.9719,35.48,12,13,1.94,NG,1738603493


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
120,coahuayana de hidalgo,MX,18.7000,-103.6583,31,
201,kottapalli,IN,18.4833,79.1000,31,
531,palasa,IN,18.7667,84.4167,75,
543,san luis de la loma,MX,17.2706,-100.8939,54,
551,beli,NG,7.8587,10.9719,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lon = row["Lat"], row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coahuayana de hidalgo - nearest hotel: Hotel Águila Real
kottapalli - nearest hotel: Hotel Nawab's
palasa - nearest hotel: Haritha Beach Resort
san luis de la loma - nearest hotel: Villa Don Manuel
beli - nearest hotel: Seven sisters central guest inn Bali


,City,Country,Lat,Lng,Humidity,Hotel Name
120,coahuayana de hidalgo,MX,18.7000,-103.6583,31,Hotel Águila Real
201,kottapalli,IN,18.4833,79.1000,31,Hotel Nawab's
531,palasa,IN,18.7667,84.4167,75,Haritha Beach Resort
543,san luis de la loma,MX,17.2706,-100.8939,54,Villa Don Manuel
551,beli,NG,7.8587,10.9719,12,Seven sisters central guest inn Bali


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
%%capture --no-display

# Configure the map plot
ideal_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    size="Humidity",
    hover_cols=["Hotel Name", "Country"],
    tiles="OSM",
    color="City",
    alpha=0.6,
    width=1000,
    height=600
)

# Display the map
ideal_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)